# ECE 364 Lecture 8
## Automatic Differentiation II: Backpropagation

## Lecture goals
After this lecture, you should be able to:
* Depict the computational graph of toy multivariable functions.
* Trace the forward pass of a function's computational graph.
* Perform the backward pass of a computational graph by hand to compute the partial derivatives of all variables.
* Use the ``torch.autograd``engine to perform backpropagation through a computational graph.

## Computational Graphs

Recall from previous lectures that we may break apart complicated functions as a composition of simple functions. For example,

$$
\begin{align*}
f(x, y) &= x^2\cos(xy)+1\\
f(x, y) &= f_1(f_2(f_3(x,y),f_4(f_5(x, y)))))\\
f_5(x, y) &= xy\\
f_4(f_5) &= \cos(f_5)\\
f_3(x, y) &= x^2\\
f_2(f_3, f_4) &= f_3f_4\\
f_1(f_2) &= f_2+1\\
f(x, y) &= f_1
\end{align*}
$$

This decomposition was helpful to systematically compute (partial) derivatives via chain rule. In this lecture, we will see how such a depiction of multivariable functions will allow us to efficiently and exactly compute partial derivatives (and by definition gradients). The above $f(x, y)$ may be represented as a **directed acyclic graph** (DAG) also referred to as a computational graph. Each node in the graph represents intermediate results of computation or values, while edges define functions applied to the values at each node.

For the above $f(x, y)$, we obtain the following computational graph where each $w_i$ gives the result of each intermediate operation, including the input of variables $x=w_1$ and $y=w_2$.

<div>
<center><img src="computational_graph.png" width="800"/> </center>
</div>

According to this graph, we have defined
$$
\begin{align*}
w_1 &=x\\
w_2 &=y\\
w_3 &= w_1^2\\
w_4 &= w_1w_2\\
w_5 &= \cos(w_4)\\
w_6 &= w_3w_5\\
w_7 &= w_6+1.
\end{align*}
$$
We can double-check our equations from the computational graph by plugging back in starting from $f(x,y)=w_7$.
$$
\begin{align*}
f(x, y) &= w_7\\
&= w_6+1\\
&= w_3w_5+1\\
&= w_1^2\cos(w_4)+1\\
&= x^2\cos(w_1w_2)+1\\
&= x^2\cos(xy)+1.
\end{align*}
$$

## Lecture Exercise (Part 1):
Consider the function
$$
g(x, y) = xye^{xy}. 
$$
Sketch the computational graph of $g(x, y)$.

**Hint**: Think of this as $g(x, y)= h(x,y)e^{h(x, y)}$.

## Backpropagation

Using computational graphs, we would like to develop a procedure for exact and efficient automatic differentiation. The procedure we will examine is known as **backpropagation**. Backpropagation is an example of auto-differentation by **reverse accumulation** wherein gradients are accumulated by traversing the computational graph backwards from every seed node. This accumulation is accomplished by two traversals or *passes* through the graph: a forward pass and a backward pass.

### Forward Pass
The forward pass through a computational graph is the direct calculation of applying the represented function to the provided inputs. Starting from every **input node**, $w_1$ and $w_2$ in our example, we follow each directed edge to the next node and perform the indicated operation to the available input(s) to generate the intermediate value at this next node. The next node then transmits this result to any of its **successor nodes** and so on until we reach any node(s) that have no successors. For the purposes of backpropagation, we refer to these end nodes with no successors as **seed nodes**. Below, we depict the forward pass through $f(x, y) = x^2\cos(xy)+1$ for $(x, y)=(2, \frac{\pi}{2})$.

<div>
<center><img src="forward_pass.png" width="800"/> </center>
</div>

### Backward Pass
After completing the forward pass, we now have exact values of the computation from each node. We also know the operation performed along each edge as each successor node may store the operation performed to obtain its result and the nodes for which it acts as the successor. For example, node $w_6$ performs the multiplication operation from nodes $w_3$ and $w_5$, i.e. $w_6=w_3w_5$. In summary, we have values at each node, operations at each node, and a directed acyclic graph structure which may be traversed backwards from each seed node.

Starting from a seed node, backpropagation works backwards to each **predecessor node** and transmits the partial derivative with respect to that predecessor node according to the stored function values and known gradient function. For example, a node that multiplies two inputs has the stored values and can easily define a gradient function with respect to each input as being the other input, i.e. $\frac{\partial w_iw_j}{\partial w_i}=w_j$. More concretely, a successor node $w_i$ will propagate the partial derivative with respect to each of its predecessor nodes to that node. Above, this could be $w_4$ transmits $\frac{\partial w_4}{\partial w_1}$ to $w_1$ and $w_4$ transmits $\frac{\partial w_4}{\partial w_2}$ to $w_2$. Again, each node has a defined forward operation, backward operation (gradient function), and stored values from the forward pass. Below, we depict each of the backpropagated partial derivatives.

<div>
<center><img src="backprop_partial.png" width="800"/> </center>
</div>

## Lecture Exercise (Part 2):
Return to your computational graph from Part 1 and label all the partial derivatives moving backwards from each node to its predecessors, i.e. like in the above example for $f(x ,y)$. For convenience, recall that 
$$
g(x, y) = xye^{xy}.
$$

## Accumulating Gradients
Propagating these partial derivatives and accumulating more complicated derivatives by chain rule at each node, however, still requires some coordination. We refer to these accumulated partial derivatives along the backward pass as the **adjoint** at each node and we denote the adjoint at node $i$ as $\bar{w}_i$. The **adjoint** at each node is calculated by

$$
\begin{align*}
    \bar{w}_i &= \frac{\partial f}{\partial w_i}\\
    \bar{w}_i &= \sum_{j\in\textrm{successors}(w_i)}\bar{w}_j\frac{\partial w_j}{\partial w_i}.
\end{align*}
$$

Let's compute the adjoint values for the above computational graph to gain some intuition for the above equations. We first have $\bar{w}_7$ as the "base case" since $f(x, y)=w_7$; thus,
\begin{align*}
    \bar{w}_7 &= \frac{\partial f}{\partial w_7}\\
             &=1.
\end{align*}

Next, $w_7$ backpropagates to its predecessor $w_6$:
\begin{align*}
    \bar{w}_6 &= \bar{w}_7\frac{\partial w_7}{\partial w_6}\\
    &= \frac{\partial f}{\partial w_7}\frac{\partial w_7}{\partial w_6}\\
    &= 1\\
\end{align*}
Then, $w_6$ backpropagates to its predecessors $w_5$ and $w_3$:
\begin{align*}
    \bar{w}_5 &= \bar{w}_6\frac{\partial w_6}{\partial w_5}\\
    &= \frac{\partial f}{\partial w_6}\frac{\partial w_6}{\partial w_5}\\
    &= w_3\\
    \bar{w}_3 &= \bar{w}_6\frac{\partial w_6}{\partial w_3}\\
     &= \frac{\partial f}{\partial w_6}\frac{\partial w_6}{\partial w_3}\\
     &= w_5.
\end{align*}
And so on! The below figure shows all of these adjoint quantities in green.

<div>
    <center><img src="backprop_adjoint.png" width="800"/></center>
</div>

Of note, we see that
\begin{align*}
    \bar{w}_1 &= \bar{w}_3\frac{\partial w_3}{\partial w_1} + \bar{w}_4\frac{\partial w_4}{\partial w_1}\\
    &= \frac{\partial f}{\partial w_3}\frac{\partial w_3}{\partial w_1}+\frac{\partial f}{\partial w_4}\frac{\partial w_4}{\partial w_1}\\
    &= 2w_1w_5-w_2w_3\sin(w_4)
\end{align*}
since $w_1$ has two successor nodes and thus $\frac{\partial f}{\partial w_1}$ backpropagates through two separate paths that depend on $w_1$.

## Lecture Exercise (Part 3):
Perform backpropagation by computing and labeling the adjoints, i.e. each of the $\bar{w}_i=\frac{\partial g}{\partial w_i}$ values, for your computational graph of $g(x,y)$ from Parts 1 and 2.

**Hint**: Assuming we have $w_1=x$ and $w_2=y$, we know that $\bar{w}_1=\frac{\partial g}{\partial x}$ and $\bar{w}_2=\frac{\partial g}{\partial y}$. You can verify your results by making sure $\bar{w}_1$ and $\bar{w}_2$ match the below hand-calculated partial derivatives:
$$
\begin{align*}
g(x,y) &= xye^{xy}\\
\frac{\partial g}{\partial x} &= ye^{xy}+xy^2e^{xy}\\
\frac{\partial g}{\partial y} &= xe^{xy}+x^2ye^{xy}.
\end{align*}
$$

## And finally (drumroll)...
Altogether, we arrive at our final backpropagation results and final partial derivatives at the input nodes $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial y}$.

<div>
    <center><img src="backprop_full.png" width="800"/></center>
</div>

The entire procedure of backpropagation only requires one forward pass through the computational graph to establish the values at each node and one backward pass to accumulate gradients from seed nodes back through the entire graph. The backward pass is made significantly more efficient by the computed adjoint values that represent the accumulated gradients up to that node via chain rule. Each predecessor node re-uses the adjoint values of its successors and accumulates the partial derivatives of its successors with respect to itself. Thus, backpropagation may be seen as a form of **dynamic programming** as we recursively re-use previous computation for the next iteration or step of the algorithm.

### Is this the only way to do gradient accumulation?
The answer is no! Backpropagation is an example of reverse accumulation of gradients and there is, in fact, forward accumulation versions that also utilize chain rule to compute gradients in computational graphs. PyTorch and any other deep learning framework implement backpropagation for automatic differentiation; however, the interested reader can refer to [this Wikipedia article](https://en.wikipedia.org/wiki/Automatic_differentiation#Automatic_differentiation_using_dual_numbers) for information on how dual numbers are used for forward-mode accumulation.

## Comparing to the ``torch.autograd`` Engine

We explored the use of PyTorch to automatically compute derivatives in an earlier lecture, but now we can more precisely inspect the partial derivatives at each node in the computational graph using our knowledge of backpropagation. The below code implements every step of the computational graph of $f(x, y)=x^2\cos(xy)+1$ and compares our hand-calculated adjoint values to the backpropagation values generated by PyTorch.

In [1]:
import torch
import numpy as np

x = torch.tensor([float(-1)]) # make sure gradients are computed when backpropagation is called
y = torch.tensor([np.pi/3])
x.requires_grad = True
y.requires_grad = True

w1 = x
w2 = y
w3 = w1**2
w4 = w1*w2
w5 = torch.cos(w4)
w6 = w3*w5
w7 = w6+1
f = w7

# manual gradients
with torch.no_grad():
    # adjoints
    w7bar = 1
    w6bar = 1
    w5bar = w3
    w4bar = -w3*torch.sin(w4)
    w3bar = w5
    w2bar = -w1*w3*torch.sin(w4)
    w1bar = 2*w1*w5 - w2*w3*torch.sin(w4)
    
# automatic gradients via backpropagation
w3.retain_grad(), w4.retain_grad(), w5.retain_grad(), w6.retain_grad(), w7.retain_grad() # making sure PyTorch populates all gradients
f.backward() # initiate backpropagation from f as the seed node

print('Comparing our calculations to PyTorch Autograd:')
print('w1: Manual = {}, PyTorch = {}'.format(w1bar, w1.grad))
print('w2: Manual = {}, PyTorch = {}'.format(w2bar, w2.grad))
print('w3: Manual = {}, PyTorch = {}'.format(w3bar, w3.grad))
print('w4: Manual = {}, PyTorch = {}'.format(w4bar, w4.grad))
print('w5: Manual = {}, PyTorch = {}'.format(w5bar, w5.grad))
print('w6: Manual = {}, PyTorch = {}'.format(w6bar, w6.grad))
print('w7: Manual = {}, PyTorch = {}'.format(w7bar, w7.grad))

Comparing our calculations to PyTorch Autograd:
w1: Manual = tensor([-0.0931]), PyTorch = tensor([-0.0931])
w2: Manual = tensor([-0.8660]), PyTorch = tensor([-0.8660])
w3: Manual = tensor([0.5000], grad_fn=<CosBackward0>), PyTorch = tensor([0.5000])
w4: Manual = tensor([0.8660]), PyTorch = tensor([0.8660])
w5: Manual = tensor([1.], grad_fn=<PowBackward0>), PyTorch = tensor([1.])
w6: Manual = 1, PyTorch = tensor([1.])
w7: Manual = 1, PyTorch = tensor([1.])


## Lecture Exercise (Part 4):
Use the ``torch.autograd`` engine like above to verify the adjoints and final gradient results for $h(x, y)$.

In [ ]:
import torch
import numpy as np

x = torch.tensor([1.], requires_grad=True) # make sure gradients are computed when backpropagation is called
y = torch.tensor([2.], requires_grad=True)

# create each w_i node in your graph
w_1 = # fill in these lines
w_2 =
w_3 = 
w_4 = 
w_5 = 
h = w_5
# manual gradients
with torch.no_grad():
    # fill in your adjoints Part 3
    w1bar = # fill in these lines
    w2bar = 
    w3bar = 
    w4bar = 
    w5bar = 
# automatic gradients via backpropagation
w3.retain_grad(), w4.retain_grad(), w5.retain_grad()
h.backward()

# print comparisons
print('Comparing our calculations to PyTorch Autograd:')
print('w1: Manual = {}, PyTorch = {}'.format(w1bar, w1.grad))
print('w2: Manual = {}, PyTorch = {}'.format(w2bar, w2.grad))
print('w3: Manual = {}, PyTorch = {}'.format(w3bar, w3.grad))
print('w4: Manual = {}, PyTorch = {}'.format(w4bar, w4.grad))
print('w5: Manual = {}, PyTorch = {}'.format(w5bar, w5.grad))

## Next time: Utilizing the ``torch.autograd`` engine for optimization

In part 3 of our auto-differentation lectures, we will connect our knowledge from part 1 about gradient descent and function optimization with the auto-differentiation of this lecture to let PyTorch automatically perform optimization problems like finding local minima of a function and even perform function approximation from toy datasets. There is much much more to be said and utilized from the ``torch.autograd`` and ``torch.optim`` modules. We will explore some next time and more later in this course!